In [1]:
%cd /home/joe/hack104-rec

from hack104_rec import TrainClick, TrainClickProcessed, score_relevance, ndcg_at_k, TrainClickExploded, TrainClickCTR
from hack104_rec import TrainActionProcessed, TrainActionCount, TrainActionUnstacked, JobDateRange, TrainClickGrouped
from hack104_rec import udfy
import datetime as dt
from pyspark.sql.types import ArrayType, ShortType, StructType, StructField, LongType, ShortType
import xgboost as xgb
from hack104_rec.model import series_cv, query_dataset, get_y_X
import pyspark
from hack104_rec import ndcg_score_of_prediction, ndcg_score_of_truth
from pyspark.sql import functions as f
from carriage import Stream, X

import pendulum as pdm

spark_conf = pyspark.SparkConf()
spark_conf.setAll([
    ('spark.driver.memory', '10g'),
    ('spark.executor.memory', '10g'),
    ('spark.driver.maxResultSize', '10g')
])
display(spark_conf.getAll())
spark = pyspark.sql.SparkSession.builder.config(conf=spark_conf).master('local[16]').getOrCreate()


/home/joe/hack104-rec


dict_items([('spark.driver.memory', '10g'), ('spark.executor.memory', '10g'), ('spark.driver.maxResultSize', '10g')])

# Trainset

In [2]:
cv_folds = list(series_cv(31, train_ratio=0.9))
display(cv_folds)

cv_fold = cv_folds[0]
display(cv_fold)

base_date = pdm.date(2018, 5, 1)

[CVFold(train_start=0, train_stop=20, test_start=21, test_stop=22),
 CVFold(train_start=4, train_stop=24, test_start=25, test_stop=26),
 CVFold(train_start=8, train_stop=28, test_start=29, test_stop=30)]

CVFold(train_start=0, train_stop=20, test_start=21, test_stop=22)

In [3]:
%%time
testset_df, testset_group_lens_df = query_dataset(
    start_date=base_date.add(days=cv_fold.test_start), 
    stop_date=base_date.add(days=cv_fold.test_stop),
    features_start_date=base_date.add(cv_fold.train_start),
    features_stop_date=base_date.add(cv_fold.train_stop),
    spark=spark)

test_y, test_X = get_y_X(testset_df)
test_dm = xgb.DMatrix(test_X, label=test_y)
test_dm.set_group(testset_group_lens_df['count'])


CPU times: user 19.2 s, sys: 748 ms, total: 20 s
Wall time: 55.7 s


In [4]:
%%time 
trainset_df, trainset_group_lens_df = query_dataset(
    start_date=base_date.add(days=cv_fold.train_start), 
    stop_date=base_date.add(days=cv_fold.train_stop),
    features_start_date=base_date.add(cv_fold.train_start),
    features_stop_date=base_date.add(cv_fold.train_stop),
    spark=spark)

train_y, train_X = get_y_X(trainset_df)
train_dm = xgb.DMatrix(train_X, label=train_y)
train_dm.set_group(trainset_group_lens_df['count'])

CPU times: user 3min 43s, sys: 15.2 s, total: 3min 58s
Wall time: 4min 55s


## Original nDCG

In [7]:
ndcg_score_of_truth(test_y)

0.3527401043952894

# Train

In [5]:
train_X.columns

Index(['sctp', 'mode', 'wktm', 'order', 'asc', 'scmax', 'scmin', 'kwop', 'm',
       's5', 'sr', 'web_total', 'web_viewJob', 'web_saveJob', 'web_applyJob',
       'mobileWeb_total', 'mobileWeb_viewJob', 'mobileWeb_saveJob',
       'mobileWeb_applyJob', 'app_total', 'app_viewJob', 'app_saveJob',
       'app_applyJob', 'viewJob_total', 'saveJob_total', 'applyJob_total',
       'total_total', 'open_days', 'web_total_per_day', 'web_viewJob_per_day',
       'web_saveJob_per_day', 'web_applyJob_per_day',
       'mobileWeb_total_per_day', 'mobileWeb_viewJob_per_day',
       'mobileWeb_saveJob_per_day', 'mobileWeb_applyJob_per_day',
       'app_total_per_day', 'app_viewJob_per_day', 'app_saveJob_per_day',
       'app_applyJob_per_day', 'viewJob_total_per_day',
       'saveJob_total_per_day', 'applyJob_total_per_day',
       'total_total_per_day', 'saveJob_per_viewJob', 'applyJob_per_viewJob'],
      dtype='object')

In [8]:

xgb_param = dict(
    objective='rank:pairwise', 
    eta=0.1,
    gamma = 1.0 ,
    min_child_weight = 0.1,
    max_depth=6,
#    num_round = 4,
    save_period = 0,
    eval_metric = ['ndcg', 'map', 'ndcg@20-', 'ndcg@20']
)

booster = xgb.train(
    params=xgb_param,
    dtrain=train_dm, 
    num_boost_round=5,
     evals=[(train_dm, 'train'), (test_dm, 'test')]
#            Row(dm=dm_test_random_y, name='test_random')]
)

# xgb.plot_importance(booster)

[0]	train-ndcg:0.410958	train-map:0.248235	train-ndcg@20-:0.410958	train-ndcg@20:0.410958	test-ndcg:0.411899	test-map:0.249879	test-ndcg@20-:0.411899	test-ndcg@20:0.411899
[1]	train-ndcg:0.412146	train-map:0.249577	train-ndcg@20-:0.412146	train-ndcg@20:0.412146	test-ndcg:0.413513	test-map:0.251469	test-ndcg@20-:0.413513	test-ndcg@20:0.413513
[2]	train-ndcg:0.412815	train-map:0.250486	train-ndcg@20-:0.412815	train-ndcg@20:0.412815	test-ndcg:0.413721	test-map:0.252008	test-ndcg@20-:0.413721	test-ndcg@20:0.413721
[3]	train-ndcg:0.412869	train-map:0.250555	train-ndcg@20-:0.412869	train-ndcg@20:0.412869	test-ndcg:0.413944	test-map:0.252388	test-ndcg@20-:0.413944	test-ndcg@20:0.413944
[4]	train-ndcg:0.413319	train-map:0.251003	train-ndcg@20-:0.413319	train-ndcg@20:0.413319	test-ndcg:0.414251	test-map:0.25274	test-ndcg@20-:0.414251	test-ndcg@20:0.414251


## Prediction nDCG

In [10]:
pred_y = booster.predict(test_dm)
ndcg_score_of_prediction(test_y, pred_y)

0.4216748954315162

# Train Click

In [2]:
%%time
TrainClickProcessed.populate(spark)

CPU times: user 52.8 ms, sys: 12.4 ms, total: 65.2 ms
Wall time: 20.3 s


In [3]:
TrainClickProcessed.calc_ndcg()

0.35201341909614575

In [6]:
%%time
TrainClickGrouped.populate()


CPU times: user 41.3 ms, sys: 12.6 ms, total: 53.8 ms
Wall time: 17.2 s


In [7]:
TrainClickGrouped.calc_ndcg()

0.3520389673729538

In [7]:
%%time
TrainClickExploded.populate()

CPU times: user 14.9 ms, sys: 0 ns, total: 14.9 ms
Wall time: 11.6 s


In [14]:
TrainClickExploded.query().show()

+------+----------+-------------------+--------------------+-------------+-----------+--------+---+
|source|      date|           datetime|        query_params|          gid|pos_in_list|     job|rel|
+------+----------+-------------------+--------------------+-------------+-----------+--------+---+
|   web|2018-05-01|2018-05-01 01:03:10|[,, 1,, [17060337...|1520418422784|          0|10186727|  0|
|   web|2018-05-01|2018-05-01 01:03:10|[,, 1,, [17060337...|1520418422784|          1|10220146|  0|
|   web|2018-05-01|2018-05-01 01:03:10|[,, 1,, [17060337...|1520418422784|          2|10270447|  0|
|   web|2018-05-01|2018-05-01 01:03:10|[,, 1,, [17060337...|1520418422784|          3| 7794353|  1|
|   web|2018-05-01|2018-05-01 01:03:10|[,, 1,, [17060337...|1520418422784|          4|10290761|  0|
|   web|2018-05-01|2018-05-01 01:03:10|[,, 1,, [17060337...|1520418422784|          5|10295706|  0|
|   web|2018-05-01|2018-05-01 01:03:10|[,, 1,, [17060337...|1520418422784|          6|10286548|  0|


In [2]:
%%time
TrainClickCTR.populate()

AnalysisException: "cannot resolve '`action`' given input columns: [source, query_params, click, pos_in_list, job, gid, date, rel, datetime];;\n'Aggregate [date#1, source#0, 'action, query_params#3.keyword, 'job_in_list], [date#1, source#0, 'action, query_params#3.keyword AS keyword#40, 'job_in_list, count(1) AS impr#36L, sum(click#16L) AS click#38L]\n+- AnalysisBarrier\n      +- Project [source#0, date#1, datetime#2, query_params#3, gid#4L, pos_in_list#5, job#6L, rel#7L, rel#7L AS click#16L]\n         +- Relation[source#0,date#1,datetime#2,query_params#3,gid#4L,pos_in_list#5,job#6L,rel#7L] parquet\n"

# Train Action

In [2]:
%%time
TrainActionProcessed.populate()

CPU times: user 31.9 ms, sys: 15.8 ms, total: 47.7 ms
Wall time: 1min 10s


In [2]:
%%time
TrainActionCount.populate()

CPU times: user 27.7 ms, sys: 15.5 ms, total: 43.1 ms
Wall time: 30.2 s


In [2]:
%%time
TrainActionUnstacked.populate()

# Job

In [2]:
%%time
JobDateRange.populate()

CPU times: user 34.7 ms, sys: 20.8 ms, total: 55.5 ms
Wall time: 14.2 s


In [11]:
date_start, date_stop = JobDateRange.query().select(f.min('date_start'), f.max('date_stop')).first()

In [19]:
(date_stop - date_start).days

31

$\frac{length}{1+R\cdot(n-1)}$

# label

In [344]:
train_click_sdf = spark.read.parquet('data/train-click.pq')
(train_click_sdf
 .select('*', 
         f.posexplode('joblist').alias('pos', 'job'),
        )
 .withColumn('action', f.when(f.col('job') == f.col('jobno'), f.col('action')).otherwise(None))
 .withColumn('keyword', f.col('query_params.keyword'))
 .drop('query_params')
 .drop('querystring')
 .drop('joblist')
 .drop('jobno')
).write.parquet('data/train-label.pq', mode='overwrite')


In [12]:
train_label_sdf = spark.read.parquet('data/train-label.pq')
train_label_sdf.printSchema()

root
 |-- action: string (nullable = true)
 |-- date: date (nullable = true)
 |-- source: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: long (nullable = true)
 |-- pos: integer (nullable = true)
 |-- job: long (nullable = true)
 |-- keyword: string (nullable = true)



In [18]:
train_label_sdf.show(40)

+--------+----------+------+-------------------+--------+-----------+---+--------+-------+
|  action|      date|source|           datetime|  tokens|         id|pos|     job|keyword|
+--------+----------+------+-------------------+--------+-----------+---+--------+-------+
|    null|2018-05-29|   web|2018-05-29 19:34:16|[证券, 承销]|60129542144|  0| 7825266|   證券承銷|
|    null|2018-05-29|   web|2018-05-29 19:34:16|[证券, 承销]|60129542144|  1| 7847926|   證券承銷|
|    null|2018-05-29|   web|2018-05-29 19:34:16|[证券, 承销]|60129542144|  2| 8997950|   證券承銷|
|    null|2018-05-29|   web|2018-05-29 19:34:16|[证券, 承销]|60129542144|  3| 6305216|   證券承銷|
|    null|2018-05-29|   web|2018-05-29 19:34:16|[证券, 承销]|60129542144|  4|10427119|   證券承銷|
|    null|2018-05-29|   web|2018-05-29 19:34:16|[证券, 承销]|60129542144|  5| 6742728|   證券承銷|
|    null|2018-05-29|   web|2018-05-29 19:34:16|[证券, 承销]|60129542144|  6| 7325695|   證券承銷|
|    null|2018-05-29|   web|2018-05-29 19:34:16|[证券, 承销]|60129542144|  7|10152693|   證券承銷|

In [324]:
train_action_count_sdf = spark.read.parquet('data/train-action-count.pq')
action_count_per_day = (train_action_count_sdf
 .groupby('jobno', 'action')
 .agg(f.min('date').alias('date_start'),
      f.max('date').alias('date_stop'), 
      f.sum('count').alias('count'))
 .withColumn('date_diff', f.datediff('date_stop', 'date_start') + 1)
 .withColumn('count_per_day', f.col('count') / f.col('date_diff'))
 .select('jobno', 'action', 'count_per_day')
  .groupby('jobno')
 .pivot('action', ['viewJob', 'saveJob', 'applyJob'])
 .mean('count_per_day')
.fillna(0.0)

  .write.parquet('data/action-count-per-day.pq', mode='overwrite') )


In [3]:
action_count_per_day = spark.read.parquet('data/action-count-per-day.pq')
action_count_per_day.show()

+--------+------------------+-------------------+-------------------+
|   jobno|           viewJob|            saveJob|           applyJob|
+--------+------------------+-------------------+-------------------+
| 5685414| 4.290322580645161|                0.0|                0.0|
| 6951559|18.870967741935484| 0.2413793103448276|0.35714285714285715|
| 7108361|22.333333333333332| 1.1538461538461537| 0.9230769230769231|
| 7522686|3.7419354838709675|                0.0|0.10526315789473684|
| 8325514| 6.580645161290323|              0.375|             0.8125|
| 8795443| 8.225806451612904| 0.6785714285714286| 0.5862068965517241|
| 9370683| 11.32258064516129| 0.5714285714285714|                0.6|
| 9666534|13.333333333333334|               0.75| 0.6666666666666666|
| 9912410|25.071428571428573| 0.6521739130434783| 1.3703703703703705|
| 9915543| 6.290322580645161| 0.4827586206896552| 0.7142857142857143|
| 8081373|               8.0| 0.6153846153846154| 0.2222222222222222|
| 9301703|21.2142857

In [4]:
train_label_with_action_sdf = train_label_sdf.join(action_count_per_day
                                                   .withColumnRenamed('jobno', 'job'), 
                                                   on='job', how='left')

In [5]:
train_label_with_action_sdf.rdd.getNumPartitions()

16

In [17]:
train_label_with_action_sdf.show()

NameError: name 'train_label_with_action_sdf' is not defined

In [6]:
train_label_with_action_df = (train_label_with_action_sdf
 .withColumn('label', f.when(f.col('action') == 'clickJob', 1)
             .when(f.col('action') == 'saveJob', 2)
             .when(f.col('action')== 'applyJob', 3)
             .otherwise(0))
 .select('label', 'id', 'pos', 'viewJob', 'saveJob', 'applyJob')
).toPandas()

In [21]:

from carriage import StreamTable, Stream, X
import numpy as np

In [25]:
train_groups = np.array(Stream(train_label_with_action_df.itertuples())
                        .group_by_as_stream(X.id)
                        .map(lambda row: row.values.len())
                        .to_list()
                        )

In [28]:
train_y = train_label_with_action_df['label'].values

In [34]:
train_X = train_label_with_action_df.loc[:, 'viewJob':].values

In [35]:
dm_train = xgb.DMatrix(train_X, label=train_y)
dm_train.set_group(train_groups)


In [77]:

xgb_param = dict(
    objective='rank:pairwise', 
    eta=0.1,
    gamma = 1.0 ,
    min_child_weight = 0.1,
    max_depth=6,
#    num_round = 4,
    save_period = 0,
    eval_metric = 'map'
)

booster = xgb.train(
    params=xgb_param,
    dtrain=dm_train, 
    num_boost_round=20,
     evals=[(dm_train, 'train')]
#            Row(dm=dm_test_random_y, name='test_random')]
)

# xgb.plot_importance(booster)

[0]	train-map:0.285321
[1]	train-map:0.285664
[2]	train-map:0.285698
[3]	train-map:0.286132
[4]	train-map:0.28632
[5]	train-map:0.286385
[6]	train-map:0.286501
[7]	train-map:0.286706
[8]	train-map:0.286703
[9]	train-map:0.28674
[10]	train-map:0.286798
[11]	train-map:0.286876
[12]	train-map:0.286923
[13]	train-map:0.286954
[14]	train-map:0.287018
[15]	train-map:0.287001
[16]	train-map:0.286997
[17]	train-map:0.287015
[18]	train-map:0.287036
[19]	train-map:0.287033


# nDCG

In [76]:
train_predict_y = booster.predict(dm_train)
import numpy as np
Stream(np.split(train_predict_y, train_groups.cumsum())).map(lambda labels: ndcg_at_k(labels, len(labels))).mean()

0.8472356442900008

In [78]:
from sklearn.metrics import average_precision_score

In [93]:
(Stream(np.split(train_y, train_groups.cumsum()))
 .map(lambda labels: average_precision_score(labels, list(reversed(range(len(labels))))))
#  .map(lambda labels: ndcg_at_k(labels, len(labels), method=1))
 .mean()
)

/home/joe/hack104-rec/.venv/lib/python3.6/site-packages/sklearn/metrics/ranking.py:444: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


ValueError: Data is not binary and pos_label is not specified

In [ ]:
y_groups

# job

In [284]:
job_sdf = spark.read.json('data/job.json')
job_sdf.show(vertical=True)


-RECORD 0---------------------------
 addr_no     | 6001005004           
 custno      | 7c01228e-5bc6-479... 
 description | ※輔導國小課業內容。
※中、英語... 
 edu         | 56                   
 exp_jobcat1 | 0                    
 exp_jobcat2 | 0                    
 exp_jobcat3 | 0                    
 industry    | 1005001009           
 job         | 國小安親課輔老師             
 jobcat1     | 2016002008           
 jobcat2     | 2016002011           
 jobcat3     | 0                    
 jobno       | 4371531              
 language1   | 14444                
 language2   | 1111                 
 language3   | 1111                 
 major_cat   | 0                    
 major_cat2  | 0                    
 major_cat3  | 0                    
 need_emp    | 1                    
 need_emp1   | 2                    
 others      | 1.提供勞健保
2.待遇優
3.需... 
 period      | 2                    
 role        | 1                    
 role_status | 2049                 
 s2          | 4                    
 s